In [1]:
import re
import nltk
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
import matplotlib as mpl
from sklearn.manifold import MDS

Código inspirado (declaradamente) em:  
http://dogdogfish.com/2015/05/11/finding-topics-in-harry-potter-using-k-means-clustering/  
https://github.com/Kali89/HarryPotterClusters  
http://brandonrose.org/clustering  

In [2]:
class HarryPotter(object):

    def __init__(self):
        self.harry_potter_titles = [u'1_Harry_Potter_Sorcerer_s_Stone.txt',
                                    u'2_Harry_Potter_The_Chamber_Of_Secrets.txt',
                                    u'3_Harry_Potter_Prisoner_of_Azkaban.txt', 
                                    u'4_Harry_Potter_The_Goblet_of_Fire.txt',
                                    u'5_Harry_Potter_Order_of_the_Phoenix.txt', 
                                    u'6_Harry_Potter_The_Half_Blood_Prince.txt', 
                                    u'7_Harry_Potter_Deathly_Hollows.txt']


        self.chapters_regex = re.compile('^CHAPTER\s[0-9A-Z]+\s-\s.*?\n')
        self.chapter_list = []
        self.chapter_texts = []
        self.book_list = []
        self.clusters = None
        self.tfidf_vectorizer = TfidfVectorizer(max_df = 0.75, max_features = 4000000, min_df = 0.05, stop_words='english', use_idf=True, tokenizer=self.tokenize, ngram_range=(1,4), decode_error='ignore')
        self.NUM_CLUSTERS = 25
        self.km = KMeans(n_clusters = self.NUM_CLUSTERS)
        self.tfidf_matrix = None
        self.terms = None
        self.chapter_frame = None
        self.cluster_colours = {u'1_Harry_Potter_Sorcerer_s_Stone' : '#7fc97f', 
                                u'2_Harry_Potter_The_Chamber_Of_Secrets' : '#beaed4', 
                                u'3_Harry_Potter_Prisoner_of_Azkaban' : '#fdc086', 
                                u'4_Harry_Potter_The_Goblet_of_Fire' : '#ffff99', 
                                u'5_Harry_Potter_Order_of_the_Phoenix.txt': '#386cb0', 
                                u'6_Harry_Potter_The_Half_Blood_Prince' : '#f0027f', 
                                u'7_Harry_Potter_Deathly_Hollows' : '#bf5b17'}

    def split_into_chapters(self, book):
        transform = lambda x: ' '.join(x.split('-')[1:]).strip().lower().decode('utf-8', 'ignore').encode('ascii', 'ignore')
        current_text = ""
        current_chapter_name = ""
        chapter_dictionary = {}
        for line in book:
            if re.match(self.chapters_regex, line):
                chapter_dictionary[current_chapter_name] = current_text.decode('utf-8', 'ignore').encode('ascii', 'ignore')
                current_chapter_name = transform(line)
                current_text = ""
            else:
                current_text += ' ' + line
        chapter_dictionary[current_chapter_name] = current_text.decode('utf-8', 'ignore').encode('ascii', 'ignore')
        ## Remove first blank bit
        del chapter_dictionary['']
        return chapter_dictionary

    def generate_chapter_and_text_lists(self):
        books_as_lists = [open(entry, 'rb').readlines() for entry in self.harry_potter_titles]
        chapter_dictionaries = dict((book_title.rstrip('txt').rstrip('.'), self.split_into_chapters(book)) 
                                    for book, book_title in zip(books_as_lists, self.harry_potter_titles))
        chapter_lookup_dictionary = dict((chapter_title, book_title) for book_title, book_dict in chapter_dictionaries.items() for chapter_title in book_dict.keys())
        for book_title, book_dict in chapter_dictionaries.items():
            for chapter_title, chapter_text in book_dict.items():
                self.chapter_list.append(chapter_title)
                self.chapter_texts.append(chapter_text)
                self.book_list.append(book_title)

    def tokenize(self, text):
        tokens = [word.lower() for sent in nltk.sent_tokenize(text.decode('utf8', 'ignore')) for word in nltk.word_tokenize(sent)]
        filtered_tokens = []
        for token in tokens:
            if re.search('[a-zA-Z]', token) and "'" not in token:
                filtered_tokens.append(token)
        return filtered_tokens

    def perform_tfidf(self):
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.chapter_texts)
        self.terms = self.tfidf_vectorizer.get_feature_names()

    def perform_clustering(self, number_of_clusters):
        self.km.fit(self.tfidf_matrix)
        self.clusters = self.km.labels_.tolist()
        chapters = {'chapter' : self.chapter_list, 'text' : self.chapter_texts, 'cluster' : self.clusters, 'book' : self.book_list}
        self.chapter_frame = pd.DataFrame(chapters, index = [self.clusters], columns = ['chapter', 'cluster', 'text', 'book'])

    def print_clusters(self, num_terms=10):
        order_centroids = self.km.cluster_centers_.argsort()[:,::-1]
        for i in range(self.NUM_CLUSTERS):
            print "Cluster %d words: " % i
            top_words = [str(self.terms[term_index]) for term_index in order_centroids[i, :num_terms]]
            print "Top words: %s" % ','.join(top_words)
            print ""
            for book, title in zip(self.chapter_frame.ix[i]['book'], self.chapter_frame.ix[i]['chapter']):
                print "Chapter: %s, Book: %s" % (title, book)
            print ""
            
    def generate_cluster_plot_frame(self):
        MDS()
        mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
        dist = 1 - cosine_similarity(self.tfidf_matrix)
        pos = mds.fit_transform(dist)
        xs, ys = pos[:,0], pos[:,1]

        self.cluster_plot_frame = pd.DataFrame(dict(x=xs, y=ys, label=self.clusters, chapter=self.chapter_list, book=self.book_list))


    def plot_all_clusters(self):
        groups = self.cluster_plot_frame.groupby('book')
        fig, ax = plt.subplots(figsize=(17,9))
        ax.margins(0.05)

        for name, group in groups:
            ax.plot(group.x, group.y, marker='o', linestyle='', ms=8, color=self.cluster_colours[name], mec='none', label=name)
            ax.set_aspect('auto')
            ax.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off')
            ax.tick_params(axis='y', which='both', left='off', top='off', labelleft='off')

        ax.legend(numpoints=1)
        for i in range(len(self.cluster_plot_frame)):
            ax.text(self.cluster_plot_frame.ix[i]['x'], self.cluster_plot_frame.ix[i]['y'], self.cluster_plot_frame.ix[i]['chapter'], size=8)

        plt.savefig('all_clusters.png')
        plt.close()

    def factors(self, n):
        return list(set(reduce(list.__add__, ([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0))))

    def median(self, factor_list):
        lst = sorted(factor_list)
        return lst[((len(lst) + 1)/2) - 1]

    def plot_all_subplots(self):
        groups = self.cluster_plot_frame.groupby('label')
        factors = self.factors(self.NUM_CLUSTERS)
        if len(factors) % 2 == 0:
            flipflop = False
            while len(factors) > 2:
                if flipflop:
                    factors.pop()
                else:
                    factors.pop(0)
                flipflop = not flipflop
            height, width = factors
        else:
            squared_factor = self.median(factors)
            height, width = squared_factor, squared_factor

        fig = plt.figure(figsize=(17,9))
        fig.suptitle("Individual clusters")
        for name, group in groups:
            ax = plt.subplot(height, width, name + 1)
            ax.set_title("Cluster %d" % (name+ 1))
            mini_group = group.groupby('book')
            for namey, mini_df in mini_group:
                ax.plot(mini_df.x, mini_df.y, marker='o', linestyle='', ms=6, color=self.cluster_colours[namey], mec='none', label=namey)
            ax.set_aspect('auto')
            ax.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off')
            ax.tick_params(axis='y', which='both', left='off', top='off', labelleft='off')
            for index in mini_df.index:
                ax.text(mini_df.ix[index]['x'], mini_df.ix[index]['y'], mini_df.ix[index]['chapter'], size=9)
        plt.savefig('all_clusters_subplot.png')
        plt.close()

    def plot_sequential_graphs(self):
        groups = self.cluster_plot_frame.groupby('label')
        for name, group in groups:
            fig, ax = plt.subplots(figsize=(17,9))
            ax.margins(0.05)
            ax.set_title("Cluster %d" % (name + 1))
            mini_group = group.groupby('book')
            for namey, mini_df in mini_group:
                ax.plot(mini_df.x, mini_df.y, marker='o', linestyle='', ms=12, color=self.cluster_colours[namey], mec='none', label=namey)
            ax.set_aspect('auto')
            ax.tick_params(axis='x',which='both',bottom='off',top='off',labelbottom='off')
            ax.tick_params(axis='y', which='both', left='off', top='off', labelleft='off')
            for index in group.index:
                ax.text(group.ix[index]['x'], group.ix[index]['y'], group.ix[index]['chapter'], size=9)
            ax.legend(numpoints=1)
            plt.savefig("Cluster-%d" % name)
            plt.close()

    def run_all(self):
        self.generate_chapter_and_text_lists()
        self.perform_tfidf()
        self.perform_clustering(self.NUM_CLUSTERS)
        self.print_clusters()
        self.generate_cluster_plot_frame()
        self.plot_all_clusters()
        self.plot_all_subplots()
        self.plot_sequential_graphs()

In [16]:
myThing = HarryPotter()

In [17]:
myThing.generate_chapter_and_text_lists()

In [18]:
books_as_lists = [open(entry, 'rb').readlines() for entry in myThing.harry_potter_titles]

In [31]:
books_as_lists[3]

['Harry Potter and the Goblet of Fire\n',
 'by J.K. Rowling\n',
 '\n',
 'To Peter Rowling.\n',
 'In Memory of Mr. Ridley.\n',
 'And to Susan Sladden.\n',
 'Who Helped Harry\n',
 'Out of His Cupboard.\n',
 '\n',
 '\n',
 'HARRY POTTER AND THE GOBLET OF FIRE\n',
 '\n',
 'CHAPTER ONE - THE RIDDLE HOUSE\n',
 '\n',
 '\tThe villagers of Little Hangleron still called it "the Riddle House," even though it had been many years since the Riddle family had lived there.  It stood on a hill overlooking the village, some of its windows boarded, tiles missing from its roof, and ivy spreading unchecked over its face.  Once a fine-looking manor, and easily the largest and grandest building for miles around, the Riddle House was now damp, derelict, and unoccupied.\n',
 '\tThe Little Hagletons all agreed that the old house was "creepy."  Half a century ago, something strange and horrible had happened there, something that the older inhabitants of the village still liked to discuss when topics for gossip we

In [29]:
dic = myThing.split_into_chapters(books_as_lists[4])
dic.keys()

['luna lovegood',
 'number twelve, grimmauld place',
 'the hogwarts high inquisitor',
 'the order of the phoenix',
 'educational decree number twenty four',
 'the lion and the serpent',
 'percy and padfoot',
 'the advance guard',
 'detention with dolores',
 'in the hogs head',
 'the ministry of magic',
 'professor umbridge',
 "hagrid's tale",
 'the woes of mrs. weasley',
 "the sorting hat's new song",
 'a peck of owls',
 'dudley demented',
 'the hearing',
 'noble and most ancient house of black',
 'grawp',
 "dumbledore's army"]

In [21]:
chapter_dictionaries = dict((book_title.rstrip('txt').rstrip('.'), myThing.split_into_chapters(book)) 
                                    for book, book_title in zip(books_as_lists, myThing.harry_potter_titles))
chapter_dictionaries.keys()

[u'5_Harry_Potter_Order_of_the_Phoenix',
 u'1_Harry_Potter_Sorcerer_s_Stone',
 u'2_Harry_Potter_The_Chamber_Of_Secrets',
 u'7_Harry_Potter_Deathly_Hollows',
 u'3_Harry_Potter_Prisoner_of_Azkaban',
 u'6_Harry_Potter_The_Half_Blood_Prince',
 u'4_Harry_Potter_The_Goblet_of_Fire']

In [22]:
myThing.perform_tfidf()

In [23]:
myThing.perform_clustering(myThing.NUM_CLUSTERS)

In [24]:
myThing.print_clusters()

Cluster 0 words: 
Top words: mrs. weasley,mrs.,fred,mr. weasley,george,said mrs. weasley,sirius,mr.,said mrs.,percy

Chapter: number twelve, grimmauld place, Book: 5_Harry_Potter_Order_of_the_Phoenix
Chapter: the order of the phoenix, Book: 5_Harry_Potter_Order_of_the_Phoenix
Chapter: the woes of mrs. weasley, Book: 5_Harry_Potter_Order_of_the_Phoenix
Chapter: noble and most ancient house of black, Book: 5_Harry_Potter_Order_of_the_Phoenix
Chapter: the burrow, Book: 2_Harry_Potter_The_Chamber_Of_Secrets
Chapter: the leaky cauldrum, Book: 3_Harry_Potter_Prisoner_of_Azkaban
Chapter: a very frosty christmas, Book: 6_Harry_Potter_The_Half_Blood_Prince
Chapter: an excess of phlegm, Book: 6_Harry_Potter_The_Half_Blood_Prince
Chapter: draco's detour, Book: 6_Harry_Potter_The_Half_Blood_Prince
Chapter: aboard the hogwart express, Book: 4_Harry_Potter_The_Goblet_of_Fire
Chapter: mayhem at the ministry, Book: 4_Harry_Potter_The_Goblet_of_Fire

Cluster 1 words: 
Top words: lockhart,malfoy,slyther

In [25]:
myThing.generate_cluster_plot_frame()

In [26]:
myThing.plot_all_clusters()

KeyError: u'5_Harry_Potter_Order_of_the_Phoenix'

In [28]:
myThing.plot_all_subplots()

KeyError: u'5_Harry_Potter_Order_of_the_Phoenix'